## Librairies

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2 as cv
%matplotlib inline

# Importing the Keras libraries and packages
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.preprocessing import image
from keras.utils import to_categorical
from tensorflow.keras.layers import Input, Lambda ,Dense ,Flatten ,Dropout
import tensorflow as tf
import os

#### Data loading

In [29]:
labels = pd.read_csv("ArSL_Data_Labels.csv")
labels = labels.drop(columns="#")
new_labels = [(labels["Class"][item], labels["File_Name"][item]) for item in labels.index]
images = [cv.imread("ArASL_Database_54K_Final/"+cl+"/"+f) for cl, f in new_labels]
# img = cv.imread("ArASL_Database_54K_Final/ain/AIN (1).JPG")

#### Numbers of images

In [30]:
print("numbers of images is ", len(images), labels.shape)

numbers of images is  54049 (54049, 2)


#### Some useful functions

In [31]:
def dico_size(shapes):
    dico = {}
    for shape in shapes:
        if dico.get(shape, False) == False:
            dico[shape] = 1
        else: 
            dico[shape] = dico[shape] + 1
    return dico

#### Visualize images sizes in our dataset

In [32]:
shapes = [img.shape for img in images]
shapes_sorted = sorted(shapes)
dico_size(shapes_sorted)

{(64, 64, 3): 53401, (256, 256, 3): 638, (1024, 768, 3): 10}

#### Resize all images to (64x64x3)

In [33]:
images_resized = []
for img in images:
    if img.shape != (64, 64, 3):
        img = cv.resize(img, (64, 64), interpolation = cv.INTER_AREA)
    images_resized.append(img)

In [34]:
dico_size(sorted([img.shape for img in images_resized]))

{(64, 64, 3): 54049}

In [56]:
data = np.array(images_resized)
print(data.shape)

(54049, 64, 64, 3)


In [58]:
# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
y = labels
# # Encode labels in column 'species'.
y['Class'] = label_encoder.fit_transform(y['Class'])
y = y['Class'].to_numpy()
y.shape

(54049,)

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.2)
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print("x_train", X_train.shape, "y_train", y_train.shape, "\nx_test", X_test.shape, "y_test", y_test.shape)

x_train (43239, 64, 64, 3) y_train (43239, 32) 
x_test (10810, 64, 64, 3) y_test (10810, 32)


In [66]:
#Initialising ResNet50
classifier_resnet = ResNet50(input_shape= (64, 64, 3),include_top=False,weights='imagenet')

In [67]:
#don't train existing weights for resnet50
for layer in classifier_resnet.layers:
    layer.trainable = False

In [71]:
classifier = classifier_resnet.output#head mode
classifier = Flatten()(classifier)#adding layer of flatten
classifier = Dropout(0.6)(classifier)
classifier = Dense(units=32, activation='softmax')(classifier)

model = Model(inputs = classifier_resnet.input , outputs = classifier)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
#fit the model
#it will take some time to train
history = model.fit(X_train, y_train, epochs =5, batch_size = 64,validation_data=(X_test,y_test))